<a href="https://colab.research.google.com/github/katiacardoso/SelectorHat/blob/main/SelectorHat_HP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# *Rede neural artificial (RNA) para classificação de texto utilizando        NLTK(Natural Language Toolkit)*
## *Discentes:* Katia Cardoso e Mariana Dias

Utilizamos camadas de neurônios, camada ocultas e uma abordagem de "saco de palavras" para organizar nossos dados de treinamento. A classificação de texto vem em 3 fatores: correspondência de padrões, algoritmos, redes neurais.

Precisamos de uma maneira de tokenizar palavras de forma confiável

In [ ]:
# use natural language toolkit
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
stemmer = LancasterStemmer()
import pandas as pd
import sklearn
import numpy as np

#importação dos dados
url = 'https://raw.githubusercontent.com/katiacardoso/SelectorHat/main/Characters.csv'
data = pd.read_csv(url, usecols=['Id','House','Name'], on_bad_lines='skip', delimiter=';')

'''url2 = 'https://raw.githubusercontent.com/solankiharsh/sorting_hat_classification/main/data/dataset_train.csv'
data2 = pd.read_csv(url2, usecols=['Index','Hogwarts House','First Name'], on_bad_lines='skip', delimiter=',')

data2 = data2.rename(columns = {'Index':'Id'})
data2 = data2.rename(columns = {'Hogwarts House':'House'})
data2 = data2.rename(columns = {'First Name':'Name'})

data = data.append(data2, ignore_index=True)

#data = data.replace(np.nan, 'Unknow')'''

#remoção de nulos
shape_antes=data.shape
print("antes: ", shape_antes)
data.dropna(subset=['House'], inplace=True)
shape_depois = data.shape
print("depois: ", shape_depois)

list_data = [list(row) for row in data.values]
print(list_data)

#mostra todos os dados no head
#pd.set_option('max_rows',999)
#data.info

antes:  (139, 3)
depois:  (101, 3)
[[1, 'Harry James Potter', 'Gryffindor'], [2, 'Ronald Bilius Weasley', 'Gryffindor'], [3, 'Hermione Jean Granger', 'Gryffindor'], [4, 'Albus Percival Wulfric Brian Dumbledore', 'Gryffindor'], [5, 'Rubeus Hagrid', 'Gryffindor'], [6, 'Neville Longbottom', 'Gryffindor'], [7, 'Fred Weasley', 'Gryffindor'], [8, 'George Weasley', 'Gryffindor'], [9, 'Ginevra (Ginny) Molly Weasley', 'Gryffindor'], [10, 'Dean Thomas', 'Gryffindor'], [11, 'Seamus Finnigan', 'Gryffindor'], [12, 'Lily J. Potter', 'Gryffindor'], [13, 'James Potter', 'Gryffindor'], [14, 'Sirius Black', 'Gryffindor'], [15, 'Remus John Lupin', 'Gryffindor'], [16, 'Peter Pettigrew', 'Gryffindor'], [17, 'Percy Ignatius Weasley', 'Gryffindor'], [18, '(Bill) William Arthur Weasley', 'Gryffindor'], [19, 'Charles Weasley', 'Gryffindor'], [20, 'Lee Jordan\xa0', 'Gryffindor'], [21, 'Oliver Wood', 'Gryffindor'], [22, 'Angelina Johnson', 'Gryffindor'], [23, 'Katie Bell', 'Gryffindor'], [24, 'Alicia Spinnet', '

In [ ]:
# classes of training data
training_data = list_data
print ("%s sentences in training data" % len(training_data))
i = len(training_data)

101 sentences in training data


In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']
nltk.download('punkt')
# loop para cada sentença dos dados de treinamento
for pattern in training_data:
    # tokeniza cada palavra da sentença
    w = nltk.word_tokenize(pattern[1])
    # adiciona na lista de palavras
    words.extend(w)
    # adiciona ao documento
    documents.append((w, pattern[2]))
    # adiciona para lista de classe
    if pattern[2] not in classes:
        classes.append(pattern[2])

#radicalize e transfoma cada palavra
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remova duplicatas
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

101 documents
6 classes ['Gryffindor', 'Ravenclaw', 'Durmstrang Institute', 'Beauxbatons Academy of Magic', 'Slytherin', 'Hufflepuff']
198 unique stemmed words ['madam', 'romild', 'mariett', 'luci', 'perc', 'mol', 'minerv', 'cedr', 'lupin', 'den', 'wood', 'spinnet', 'de', 'j.', 'isabel', 'flacc', 'bel', 'malfoy', 'slytherin', 'van', 'thoma', 'scorpi', 'regul', 'maxim', 'percy', 'amyc', ')', 'brown', 'greg', 'montagu', 'yaxley', 'terry', 'sybil', 'snap', 'corb', 'fri', 'pet', 'charl', 'crab', 'rowen', 'edward', 'narciss', 'olymp', 'eug', 'ol', 'corn', 'krum', 'bla', 'dean', 'herm', 'zabin', 'garrick', 'bulstrod', 'fat', 'theod', 'parvat', 'dumbl', 'jord', 'quirrel', 'august', 'marc', 'harry', 'rookwood', 'michael', 'abbot', 'dolohov', 'alb', 'longbottom', 'ginny', 'bloody', 'jean', 'fili', 'pansy', 'wald', 'arth', 'goldstein', 'newton', 'nichola', 'antonin', 'godr', 'carrow', 'angelin', 'riddl', 'goyl', 'marvolo', 'seam', 'wulfr', 'pot', 'hannah', 'edg', 'macnair', 'parkinson', 'lockhar

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# criação da lista de treinamento
training = []
output = []
#cria um vetor vazio para saida
output_empty = [0] * len(classes)

# treina o saco de palavras
for doc in documents:
    #inicializa
    bag = []
    #listagem
    pattern_words = doc[0]
    # radicaliza cada palavra
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # cria o saco de palavras
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # a saída é um '0' para cada tag e '1' para a tag atual
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

print ("# words", len(words))
print ("# classes", len(classes))

# words 198
# classes 6


In [ ]:
# amostra de treinamento/saída
i = 0
w = documents[i][0]
print ([stemmer.stem(word.lower()) for word in w])
print (training[i])
print (output[i])

['harry', 'jam', 'pot']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]


In [ ]:
sentences = ['Katia is in RaRaRa', 'Mariana is in GriGriGri']

for s in sentences:
  print (s)

Katia is in RaRaRa
Mariana is in GriGriGri


In [ ]:
#from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#pega um vectorizer com seus parametros básicos
vectorizer = CountVectorizer()
#Diz ao vectorizes ara criar o vocabulario das sentenças chamadnoo metodo fit e passando as sentenças
vectfit=vectorizer.fit(sentences)
#o vocablario é encontrado em vocabulary_
print(vectorizer.vocabulary_) #essa ordem é a propria biblioteca que cria
print('Numero de palavras diferentes/tamanho do vocabulario',len(vectorizer.vocabulary_) )

{'katia': 3, 'is': 2, 'in': 1, 'rarara': 5, 'mariana': 4, 'grigrigri': 0}
Numero de palavras diferentes/tamanho do vocabulario 6



Para obter os valores, chamar o metodo transform no vectorizes e passe o corpus(sentences) para ele. Ele vai retornar uma matriz esparsa

O metodo to array foi utilizado para coloca rna forma de vetores

---



In [ ]:
v1 = vectorizer.transform(sentences).toarray()
print(v1)

[[0 1 1 1 0 1]
 [1 1 1 0 1 0]]


In [ ]:
import numpy as np
import time

# a função sigmoide é utilizada para normalizar valores e sua derivada para medir a taxa de erro
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

#Também abaixo implementamos nossa função de saco de palavras, transformando uma frase de
#entrada em uma matriz de 0 e 1.
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [ ]:
#Codificamos nossa função de treinamento de rede neural para criar pesos
#sinápticos. Não fique muito animado, isso é principalmente multiplicação de matriz


def train(X, y, hidden_neurons=20, alpha=1, epochs=5000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)

    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))

        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break

        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)

        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))

        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))

        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update

        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"
  #vamos salvá-lo como uma estrutura json para representar nossos pesos sinápticos
    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [ ]:
X = np.array(training)
y = np.array(output)

start_time = time.time()
#Usou-se 20 neurônios em nossa camada oculta
#Esses parâmetros variam dependendo das dimensões e da forma dos dados de treinamento
train(X, y, hidden_neurons=20, alpha=0.1, epochs=5000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 101x198    Output matrix: 1x6
saved synapses to: synapses.json
processing time: 7.978027820587158 seconds


In [ ]:
# probability threshold - limiar de probabilidade
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json'
with open(synapse_file) as data_file:
    synapse = json.load(data_file)
    synapse_0 = np.asarray(synapse['synapse0'])
    synapse_1 = np.asarray(synapse['synapse1'])

# Essa funçãoclassify()é tudo o que é necessário para a classificação uma vez que
# os pesos das sinapses tenham sido calculados
def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ]
    results.sort(key=lambda x: x[1], reverse=True)
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s -> classificação: %s \n" % (sentence, return_results))
    return return_results

# Agora podemos gerar a probabilidade de uma sentença pertencer a uma (ou mais) de
# nossas classes. Isso é super rápido porque é o cálculo do produto de ponto em nossa
# funçãothink()function previamente definida.

classify("Harry James Potter", show_details=True)
classify("Harry Potter")
classify("Harry")
classify("Snape")
classify("Luna")
classify("Draco")
classify("Hagrid")
classify("Newton Scamander")


classify("Katia")
classify("Lidia")
#classify("Thais")
#classify("Mariana")
#classify("Wigor")
#classify("Andre")
#classify("Juliana")

found in bag: harry
found in bag: jam
found in bag: pot
sentence: Harry James Potter 
 bow: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
Harry James Potter -> classificação: [['Gryffindor', 0.9981147672933475]] 

Harry Potter -> classificação: [['Gryffindor', 0.9751013110268988]] 

Harry -> classificação: [['Gryffindor', 0.4097476309405711]] 

Snape -> classificação: [] 

Luna -> classificação: [['Ravenclaw', 0.8262551860562383]] 

Draco -> classificação: [] 

Hagrid -> classificação: [['Gryffindor', 0.8669836016840703]] 

Newton Scamander -> classificação: [['Hufflepuff', 0.9907215478432573]] 

Katia -> classificação: [] 

Lidia -> c

[]